In [1]:
from sklearn.metrics.pairwise import euclidean_distances

# Step 1: Read the CSV file
df = pd.read_csv("your_file.csv")

# Step 2: Identify output and feature columns
output_column = "output_column_name"  # Replace with the name of your output column
feature_columns = [col for col in df.columns if col != output_column]

# Step 3: Plot the distribution of all columns
df[feature_columns].hist(bins=20, figsize=(15, 10))
plt.tight_layout()
plt.show()

# Step 4: Create design of experiments only on feature columns
total_points = len(df)
lhs_points = lhs(len(feature_columns), samples=int(total_points * 0.1))
random_points = np.random.rand(int(total_points * 0.1), len(feature_columns))
ccd_points = ccdesign(len(feature_columns), center=(2**0.5), alpha='face', face='cci')

# Combine points from different methods
design_points = np.vstack((lhs_points, random_points, ccd_points))

# Optionally, you may want to scale the design points to match the ranges of your original data
# Example: design_points = design_points * (df[feature_columns].max() - df[feature_columns].min()) + df[feature_columns].min()

# Calculate the Euclidean distances between each design point and all points in the feature space
distances = euclidean_distances(df[feature_columns], design_points)

# Find the index of the closest point in the feature space for each design point
closest_indices = np.argmin(distances, axis=0)

# Get the actual feature values of the closest points
closest_points = df.iloc[closest_indices]

print("Closest points in the feature space:")
print(closest_points)


NameError: name 'pd' is not defined

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

# Step 1: Read the CSV file
df = pd.read_csv("your_file.csv")

# Step 2: Identify output and feature columns
output_column = "output_column_name"  # Replace with the name of your output column
feature_columns = [col for col in df.columns if col != output_column]

# Step 3: Scale the features
scaler = MinMaxScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

# Step 4: Create polynomial features with interaction terms only
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interaction = poly.fit_transform(df[feature_columns])

# Step 5: Manually add squared terms of individual features
X_squared = df[feature_columns] ** 2

# Step 6: Concatenate interaction terms and squared terms
X_poly = np.hstack((X_interaction, X_squared))

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_poly, df[output_column], test_size=0.2, random_state=42)

# Step 8: Fit a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Step 9: Evaluate the model
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

train_rmse = mean_squared_error(y_train, train_preds, squared=False)
test_rmse = mean_squared_error(y_test, test_preds, squared=False)

print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)
